In [1]:
import os
import requests
import time

import pandas as pd
import numpy as np

from tqdm import tqdm_notebook

import warnings
warnings.filterwarnings(action = 'ignore')

from bs4 import BeautifulSoup

In [2]:
# 현재 경로를 저장
current_path = os.getcwd()

# 데이터를 저장할 경로 저장
save_path = 'C:/Users/KCG/Desktop/해양과학 데이터 활용 경진대회/Data/'

In [3]:
# 연안구역 격자(3단계) 데이터 불러오기
third_grid = pd.read_csv('Data/연안구역_격자(3단계).csv', encoding = 'CP949')

# 고유 격자를 리스트 객체로 저장
grid_list = third_grid['격자번호'].unique().tolist()

print('연안구역 격자(3단계) 개수 :', len(third_grid['격자번호'].unique()), '\n')
display(third_grid.head())

연안구역 격자(3단계) 개수 : 549 



,격자번호
0,GR3_F2K12_R
1,GR3_F2K12_S
2,GR3_F2K12_T
3,GR3_F2K12_V
4,GR3_F2K12_W


## 1. 해양수산부_연속정보 염분(3분) Open API 데이터 수집

In [4]:
print('Shape of third_grid data *Before Create* :', third_grid.shape, '\n')

# 해양수산부_연속정보 염분(3분) Open API URL
url = 'http://apis.data.go.kr/1192000/apVhdService_Tgcsy3/getOpnTgcsy3?serviceKey=%2FTDcvv2SZo40y%2Flb5WgoPd1bVXJVTg5Q8ttaeN4Qxyf%2Fa3GfdZ%2Fk7vgaa2cotArtbeFtKoe38lyU9Hjtz6rOGA%3D%3D&numOfRows=31&pageNo=1&analsYmd=201908&gridCd={}'

# 격자번호에 염분 정보가 포함되어 있지 않는 경우도 있기 때문에, QGIS에서 구별해주기 위해 '염분여부' 변수를 생성해주도록 한다.
third_grid['염분여부'] = 'N'

# 각 격자의 염분을 저장하기 위해 '평균염분' 변수를 생성
third_grid['평균염분'] = 0

# 격자번호를 바꿔가면서 데이터 수집
for grid in tqdm_notebook(grid_list, desc = '염분 데이터 수집'):
    
    reponse = requests.get(url.format(grid))
    soup = BeautifulSoup(reponse.text, 'html.parser')
    
    # 'resultcode' 값 추출
    result_value = soup.find('resultcode').get_text()
    
    # 'resultcode'가 00인 경우에 염분 정보가 존재한다.
    if result_value != '00':
        continue
    
    else:
        third_grid.loc[third_grid['격자번호'] == grid, '염분여부'] = 'Y'
        
        # 각 격자의 평균염분 계산 후 저장
        slnty_value = np.mean([float(value.get_text().strip()) for value in soup.find_all('slnty')])
        
        third_grid.loc[third_grid['격자번호'] == grid, '평균염분'] = slnty_value

print()
print('Shape of third_grid data *After Create* :', third_grid.shape, '\n')
display(third_grid.head())

Shape of third_grid data *Before Create* : (549, 1) 



염분 데이터 수집:   0%|          | 0/549 [00:00<?, ?it/s]


Shape of third_grid data *After Create* : (549, 3) 



,격자번호,염분여부,평균염분
0,GR3_F2K12_R,Y,21.609613
1,GR3_F2K12_S,Y,21.631516
2,GR3_F2K12_T,Y,23.177613
3,GR3_F2K12_V,Y,21.433097
4,GR3_F2K12_W,Y,20.866968


## 2. 해양수산부_연속정보 수온(3분) Open API 데이터 수집

In [14]:
print('Shape of third_grid data *Before Create* :', third_grid.shape, '\n')

url = 'http://apis.data.go.kr/1192000/apVhdService_Tgcw3/getOpnTgcw3?serviceKey=%2FTDcvv2SZo40y%2Flb5WgoPd1bVXJVTg5Q8ttaeN4Qxyf%2Fa3GfdZ%2Fk7vgaa2cotArtbeFtKoe38lyU9Hjtz6rOGA%3D%3D&numOfRows=10&pageNo=1&analsYmd={}&gridCd={}'

# 격자번호에 수온 정보가 포함되어 있지 않는 경우도 있기 때문에, QGIS 구별해주기 위해 '수온여부' 변수를 생성해주도록 한다.
third_grid['수온여부'] = 'N'

# 각 격자의 수온을 저장하기 위해 '평균수온' 변수를 생성
third_grid['평균수온'] = 0

# 2019년 8월 데이터를 수집하기 위한 날짜 리스트 생성
date_list = [value.strftime('%Y%m%d') for value in pd.date_range(start = '20190801', end = '20190831', freq = 'D').tolist()]

for grid in tqdm_notebook(grid_list, desc = '수온 데이터 수집'):
    
    # 격자별로 2019년 8월의 수온 값을 저장하기 위한 리스트 객체 생성
    grid_water = []
    
    # 수온 API는 일자를 바꿔가며 데이터를 수집해야 한다.
    for date in date_list:
        
        reponse = requests.get(url.format(date, grid))
        soup = BeautifulSoup(reponse.text, 'html.parser')
        
        try:
            # 'resultcode' 값 추출
            result_value = soup.find('resultcode').get_text()

            # 'resultcode'가 00이 아닌 경우에는 수온 정보가 존재하지 않기 때문에 넘어가도록 한다.
            if result_value != '00':
                continue

            # 'resultcode'가 00인 경우에 수온 정보가 존재한다.
            else:
                wtem_value = float(soup.find('wtem').get_text().strip())

                # 'grid_water' 리스트에 저장
                grid_water.append(wtem_value)
                
        except:
            continue
            
    # 데이터 존재유무를 확인하기 위해 'grid_water' 리스트의 길이를 저장
    grid_water_length = len(grid_water)
    
    # 'grid_water' 리스트의 길이가 1 이상인 경우에만 해당 격자의 수온 정보가 포함되어 있음을 의미한다.
    if grid_water_length >= 1:
        third_grid.loc[third_grid['격자번호'] == grid, '수온여부'] = 'Y'
        
        # 각 격자의 평균수온 계산 후 저장
        wtem_value = np.mean(grid_water)
        third_grid.loc[third_grid['격자번호'] == grid, '평균수온'] = wtem_value
    
    time.sleep(0.5)
    
print()
print('Shape of third_grid data *After Create* :', third_grid.shape, '\n')
display(third_grid.head())

Shape of third_grid data *Before Create* : (549, 5) 



수온 데이터 수집:   0%|          | 0/549 [00:00<?, ?it/s]


Shape of third_grid data *After Create* : (549, 5) 



,격자번호,염분여부,평균염분,수온여부,평균수온
0,GR3_F2K12_R,Y,21.609613,Y,27.062632
1,GR3_F2K12_S,Y,21.631516,Y,27.296471
2,GR3_F2K12_T,Y,23.177613,Y,27.099375
3,GR3_F2K12_V,Y,21.433097,Y,27.366000
4,GR3_F2K12_W,Y,20.866968,Y,27.222222


In [20]:
third_grid.to_csv(save_path + 'oepn.csv', encoding = 'CP949', index = False)

In [17]:
len(third_grid[third_grid['수온여부'] == 'Y'])

54